## Ensure llama index installed / Not

In [5]:
! pip show llama-index


Name: llama-index
Version: 0.11.10
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: C:\Users\0201132\Documents\Projects\text_to_sql_project\venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


## Dataset download and preparation

In [3]:
import requests
import zipfile
import io
import os

def download_and_unzip(url, extract_to='.'):
    # Define headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    }

    # Send a GET request to the URL with headers
    try:
        response = requests.get(url, headers=headers, allow_redirects=True)
        response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        print(f"Response status code: {response.status_code}")
        print(f"Response headers: {response.headers}")
        print(f"Response content: {response.text[:500]}...")  # Print first 500 characters of response
        return

    # Check if the content is a zip file
    if response.headers.get('Content-Type') == 'application/zip':
        try:
            z = zipfile.ZipFile(io.BytesIO(response.content))
            z.extractall(extract_to)
            print(f"Files extracted to {os.path.abspath(extract_to)}")
            print("Extracted files:")
            for file in z.namelist():
                print(f" - {file}")
        except zipfile.BadZipFile:
            print("The downloaded content is not a valid zip file.")
    else:
        print(f"The response is not a zip file. Content-Type: {response.headers.get('Content-Type')}")
        # Save the content as is
        filename = url.split('/')[-1]
        with open(os.path.join(extract_to, filename), 'wb') as f:
            f.write(response.content)
        print(f"File saved as {filename}")

# Usage
url = "https://excelbianalytics.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip"
extract_to = "extracted_files"

download_and_unzip(url, extract_to)


Files extracted to c:\Users\0201132\Documents\Projects\text_to_sql_project\extracted_files
Extracted files:
 - 5m Sales Records.csv


In [4]:
# Read CSV

import pandas as pd
df = pd.read_csv('extracted_files/5m Sales Records.csv')

df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Region          object 
 1   Country         object 
 2   Item Type       object 
 3   Sales Channel   object 
 4   Order Priority  object 
 5   Order Date      object 
 6   Order ID        int64  
 7   Ship Date       object 
 8   Units Sold      int64  
 9   Unit Price      float64
 10  Unit Cost       float64
 11  Total Revenue   float64
 12  Total Cost      float64
 13  Total Profit    float64
dtypes: float64(5), int64(2), object(7)
memory usage: 534.1+ MB


In [7]:

# Convert the date column to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Display the first few rows to verify the change
df.head()


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,2016-03-06,517073523,2016-03-26,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,2010-04-18,380507028,2010-05-26,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,2015-01-08,504055583,2015-01-31,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,2014-01-19,954955518,2014-02-27,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,2019-04-26,970755660,2019-06-02,7027,205.70,117.11,1445453.90,822931.97,622521.93


In [8]:
df.sort_values(by='Order Date', inplace=True)

In [9]:
df['Order Date'][0]

Timestamp('2016-03-06 00:00:00')

In [11]:

# Define the split date
split_date = '2015-12-31'

# Create the master DataFrame (dates up to 31/12/2015)
master_df = df[df['Order Date'] <= split_date]

# Create the staging DataFrame (dates after 31/12/2015)
staging_df = df[df['Order Date'] > split_date]

# Reset the index for both DataFrames
master_df = master_df.reset_index(drop=True)
staging_df = staging_df.reset_index(drop=True)

# Display info about the resulting DataFrames
print("Master DataFrame:")
print(f"Shape: {master_df.shape}")
print(f"Date range: {master_df['Order Date'].min()} to {master_df['Order Date'].max()}")

print("\nStaging DataFrame:")
print(f"Shape: {staging_df.shape}")
print(f"Date range: {staging_df['Order Date'].min()} to {staging_df['Order Date'].max()}")


Master DataFrame:
Shape: (2805052, 14)
Date range: 2010-01-01 00:00:00 to 2015-12-31 00:00:00

Staging DataFrame:
Shape: (2194948, 14)
Date range: 2016-01-01 00:00:00 to 2020-09-10 00:00:00


## Database creation

In [13]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# Assuming you have your DataFrame ready and it's called 'df'
# If you have multiple DataFrames (like master_df and staging_df), you'll need to repeat this process for each

# Create a SQLite database
# This will create a new file named 'your_database.db' in your current working directory
database_name = 'Databases/AbcInc.db'
table_name = 'master'

# Create a SQLite engine
engine = create_engine(f'sqlite:///{database_name}')

# Write the DataFrame to SQLite
master_df.to_sql(table_name, engine, if_exists='replace', index=False)

# Verify that the data has been written
# This will print the first few rows of the table
with sqlite3.connect(database_name) as conn:
    print(pd.read_sql(f'SELECT * FROM {table_name} LIMIT 5', conn))



                              Region    Country Item Type Sales Channel  \
0              Australia and Oceania     Samoa    Clothes       Offline   
1  Central America and the Caribbean  Guatemala    Snacks       Offline   
2              Australia and Oceania      Tonga      Meat        Online   
3                             Europe    Belgium    Fruits       Offline   
4                 Sub-Saharan Africa    Comoros    Snacks       Offline   

  Order Priority                  Order Date   Order ID  \
0              H  2010-01-01 00:00:00.000000  244438779   
1              H  2010-01-01 00:00:00.000000  782777869   
2              C  2010-01-01 00:00:00.000000  443865954   
3              M  2010-01-01 00:00:00.000000  383569467   
4              H  2010-01-01 00:00:00.000000  880385816   

                    Ship Date  Units Sold  Unit Price  Unit Cost  \
0  2010-02-16 00:00:00.000000        3721      109.28      35.84   
1  2010-02-12 00:00:00.000000         221      152.58     

NameError: name 'stagging_df' is not defined

In [14]:
table_name = 'staging'

# Create a SQLite engine
engine = create_engine(f'sqlite:///{database_name}')

# Write the DataFrame to SQLite
staging_df.to_sql(table_name, engine, if_exists='replace', index=False)

# Verify that the data has been written
# This will print the first few rows of the table
with sqlite3.connect(database_name) as conn:
    print(pd.read_sql(f'SELECT * FROM {table_name} LIMIT 5', conn))



                              Region      Country        Item Type  \
0  Central America and the Caribbean     Barbados           Snacks   
1       Middle East and North Africa        Libya           Snacks   
2                               Asia   Kazakhstan       Vegetables   
3                               Asia      Myanmar  Office Supplies   
4  Central America and the Caribbean  Saint Lucia        Baby Food   

  Sales Channel Order Priority                  Order Date   Order ID  \
0       Offline              C  2016-01-01 00:00:00.000000  258368670   
1       Offline              M  2016-01-01 00:00:00.000000  314571893   
2       Offline              H  2016-01-01 00:00:00.000000  476569592   
3        Online              M  2016-01-01 00:00:00.000000  655412423   
4       Offline              L  2016-01-01 00:00:00.000000  813434875   

                    Ship Date  Units Sold  Unit Price  Unit Cost  \
0  2016-01-14 00:00:00.000000        5886      152.58      97.44   
1  2

## Embedding model selection